# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.60/12472/1 Dashboard: http://10.20.0.60:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,987,Frank,-0.6177748452569654,-0.44925839611658724
2000-01-01 00:00:01,1034,Ursula,0.4379919824537397,0.5622299708361336
2000-01-01 00:00:02,1028,Kevin,-0.2087681431760806,0.9590198540295447
2000-01-01 00:00:03,1030,Quinn,0.1819420904002027,-0.998414552204038
2000-01-01 00:00:04,1018,Wendy,0.23006612033067486,0.09283013595716438
2000-01-01 00:00:05,987,Oliver,-0.021771556714826668,0.41121172733843814
2000-01-01 00:00:06,954,Victor,0.20354220822299562,0.4012511172310109
2000-01-01 00:00:07,984,Patricia,-0.7455113727727423,-0.8808913840336603
2000-01-01 00:00:08,936,Quinn,-0.5109594966270683,0.3310249735934758


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1042,Charlie,-0.8108340506263945,-0.40102038362891435
2000-01-30 00:00:01,990,Victor,-0.8440082916818419,0.3509963617447529
2000-01-30 00:00:02,997,Tim,0.8570681227779828,0.6250277619557114
2000-01-30 00:00:03,916,Michael,0.1693898077318461,0.05259736311105545
2000-01-30 00:00:04,993,Xavier,0.28122902922396453,-0.35791034162397706
2000-01-30 00:00:05,1021,George,0.18560943419917186,-0.7035357788059664
2000-01-30 00:00:06,959,Kevin,-0.13504670078872194,0.4038664955370521
2000-01-30 00:00:07,955,George,0.10685584234485135,-0.5653143026008753
2000-01-30 00:00:08,1021,Alice,-0.843973450961389,0.09089702727970139


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,987,Frank,-0.617775,-0.449258
1,2000-01-01 00:00:01,1034,Ursula,0.437992,0.562230
2,2000-01-01 00:00:02,1028,Kevin,-0.208768,0.959020
3,2000-01-01 00:00:03,1030,Quinn,0.181942,-0.998415
4,2000-01-01 00:00:04,1018,Wendy,0.230066,0.092830


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,987,Frank,-0.617775,-0.449258
1,2000-01-01 00:00:01,1034,Ursula,0.437992,0.562230
2,2000-01-01 00:00:02,1028,Kevin,-0.208768,0.959020
3,2000-01-01 00:00:03,1030,Quinn,0.181942,-0.998415
4,2000-01-01 00:00:04,1018,Wendy,0.230066,0.092830


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.61 s, sys: 571 ms, total: 5.18 s
Wall time: 3.32 s


name
Alice      -0.000686
Bob         0.000638
Charlie     0.001649
Dan         0.001700
Edith      -0.000368
Frank      -0.001666
George      0.001777
Hannah      0.000557
Ingrid     -0.000903
Jerry       0.001417
Kevin       0.000109
Laura      -0.002138
Michael     0.000488
Norbert     0.000431
Oliver     -0.001100
Patricia    0.000969
Quinn       0.001478
Ray        -0.000649
Sarah       0.000398
Tim        -0.000183
Ursula      0.002119
Victor     -0.000750
Wendy      -0.001534
Xavier     -0.001645
Yvonne     -0.000770
Zelda      -0.004434
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.41 s, sys: 151 ms, total: 1.56 s
Wall time: 1.02 s


name
Alice      -0.000686
Bob         0.000638
Charlie     0.001649
Dan         0.001700
Edith      -0.000368
Frank      -0.001666
George      0.001777
Hannah      0.000557
Ingrid     -0.000903
Jerry       0.001417
Kevin       0.000109
Laura      -0.002138
Michael     0.000488
Norbert     0.000431
Oliver     -0.001100
Patricia    0.000969
Quinn       0.001478
Ray        -0.000649
Sarah       0.000398
Tim        -0.000183
Ursula      0.002119
Victor     -0.000750
Wendy      -0.001534
Xavier     -0.001645
Yvonne     -0.000770
Zelda      -0.004434
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.25 s, sys: 87.8 ms, total: 1.34 s
Wall time: 925 ms


name
Alice      -0.000686
Bob         0.000638
Charlie     0.001649
Dan         0.001700
Edith      -0.000368
Frank      -0.001666
George      0.001777
Hannah      0.000557
Ingrid     -0.000903
Jerry       0.001417
Kevin       0.000109
Laura      -0.002138
Michael     0.000488
Norbert     0.000431
Oliver     -0.001100
Patricia    0.000969
Quinn       0.001478
Ray        -0.000649
Sarah       0.000398
Tim        -0.000183
Ursula      0.002119
Victor     -0.000750
Wendy      -0.001534
Xavier     -0.001645
Yvonne     -0.000770
Zelda      -0.004434
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html